In [1]:
#Dependancies

import pandas as pd
import numpy as np

In [2]:
#Reading csv files

demographicDf = pd.read_csv("resources/demographic.csv")
labsDf = pd.read_csv("resources/labs.csv")
medicationsDf = pd.read_csv("resources/medications.csv", encoding="ISO-8859-1")

In [3]:
#Drops conditions in medicationsDf that appear less than 100 times

medValues = medicationsDf["RXDRSD1"].value_counts(dropna=False)
medicationsDf = medicationsDf[medicationsDf['RXDRSD1'].isin(medValues[medValues >= 100].index)]

In [4]:
#Replaces null values with a "Healthy" string

medicationsDf["RXDRSD1"] = medicationsDf["RXDRSD1"].fillna("Healthy")

In [5]:
#Renames the columns in medicationsDf

medicationsDf = medicationsDf.rename(columns=\
                                    {"RXDUSE": "Medications_Teken_Last_30_Days",\
                                      "RXDDRUG": "Generic_Drug_Name",\
                                      "RXDDRGID": "Generic_Drug_Code", \
                                      "RXQSEEN": "Was_Prescription_Container_Seen_By_Interviewer",\
                                      "RXDDAYS": "Duration_Taken_Medications",\
                                      "RXDRSC1": "ICD_10_CM_Code_1",\
                                      "RXDRSC2": "ICD_10_CM_Code_2",\
                                      "RXDRSC3": "ICD_10_CM_Code_3",\
                                      "RXDRSD1": "ICD_10_CM_Code_1_Description",\
                                      "RXDRSD2": "ICD_10_CM_Code_2_Description",\
                                      "RXDRSD3": "ICD_10_CM_Code_3_Description",\
                                      "RXDCOUNT":"The_Number_Of_Prescription_Medicines_Reported"
                                    })

In [7]:
#Setting up the newLab dataframe

#Isolates the SEQN column into it's own dataframe
labSEQN = labsDf[["SEQN"]]

#Sets null values to 0 and non-null values to 1
#newLab = labsDf.notnull().astype('int')

#Drops the current SEQN column which only has values of 1 currently
newLab = labsDf.drop(columns=["SEQN"])

#Brings the isolated SEQN column back into the dataframe
newLab = newLab.join(labSEQN, how='outer')

#Moves the SEQN column to the far left of our dataframe
columnList = list(newLab.columns.values)
columnList.pop(columnList.index('SEQN'))
newLab = newLab[['SEQN']+columnList]

#Drops new null values
#newLab = newLab.dropna()

#Changes 0 values to 2 values for future division
#for col in columnList:
#    newLab[col] = newLab[col].replace(0, 2, regex=True)

In [8]:
#Setting up the newMed dataframe

#Isolates the three columns we will be working with from the medicationsDf
newMed = medicationsDf[["SEQN", "Generic_Drug_Name", "ICD_10_CM_Code_1_Description"]]

#Renames the third column for readability
newMed = newMed.rename(columns={"ICD_10_CM_Code_1_Description": "Condition"})

#Replaces null perscriptions with a string value of "NONE"
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].fillna("NONE")

#Corrects false data
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].replace("99999", "NONE", regex=True)
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].replace("55555", "NONE", regex=True)
newMed["Generic_Drug_Name"] = newMed["Generic_Drug_Name"].replace("77777", "NONE", regex=True)

#Cleans up condition names
newMed["Condition"] = newMed["Condition"].replace("Hypothyroidism, unspecified", "Hypothyroidism", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Major depressive disorder, single episode, unspecified", "Major depressive disorder", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Anxiety disorder, unspecified", "Anxiety disorder", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Sleep disorder, unspecified", "Sleep disorder", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Allergic rhinitis, unspecified", "Allergic rhinitis", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Dorsalgia, unspecified", "Dorsalgia", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Chronic obstructive pulmonary disease, unspecified", "Chronic obstructive pulmonary disease", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Neuralgia and neuritis, unspecified", "Neuralgia and neuritis", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Edema, unspecified", "Edema", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Heart failure, unspecified", "Heart failure", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Cardiac arrhythmia, unspecified", "Cardiac arrhythmia", regex=True)
newMed["Condition"] = newMed["Condition"].replace("Allergy, unspecified", "Allergy", regex=True)

In [9]:
#Setting up the toplabsDf

#Creates a dataframe containing the top 15 used labs
toplabsDf = newLab[["SEQN","LBXRDW","LBXRBCSI","LBXHGB","LBXHCT","LBXMCVSI","LBXMCHSI","LBXPLTSI","LBXMPSI","LBXWBCSI","LBDBANO","LBDEONO","LBDMONO","LBDLYMNO","LBXBAPCT","LBXEOPCT","LBXNEPCT","LBXMOPCT","LBXLYPCT","LBDNENO","LBXHA","LBXHBS"]]

#Renames columns in toplabsDf for readability
toplabsDf = toplabsDf.rename(columns=\
                             {"LBXRDW":"Red cell distribution width (%)",
                             "LBXRBCSI":"Red blood cell count (million cells/uL)",
                             "LBXHGB":"Hemoglobin (g/dL)","LBXHCT":"Hematocrit (%)","LBXMCVSI":"Mean cell volume (fL)",
"LBXMCHSI":"Mean cell hemoglobin (pg)",
"LBXPLTSI":"Platelet count (1000 cells/uL)",
"LBXMPSI":"Mean platelet volume (fL)",
"LBXWBCSI":"White blood cell count (1000 cells/uL)",
"LBDBANO":"Basophils number (1000 cells/uL)",
"LBDEONO":"Eosinophils number (1000 cells/uL)",
"LBDMONO":"Monocyte number (1000 cells/uL)",
"LBDLYMNO":"Lymphocyte number (1000 cells/uL)",
"LBXBAPCT":"Basophils percent (%)",
"LBXEOPCT":"Eosinophils percent (%)",
"LBXNEPCT":"Segmented neutrophils percent (%)",
"LBXMOPCT":"Monocyte percent (%)",
"LBXLYPCT":"Lymphocyte percent (%)",
"LBDNENO":"Segmented neutrophils num (1000 cell/uL)",
"LBXHA":"Hepatitis A antibody",
"LBXHBS":"Hepatitis B Surface Antibody"
                             })

#Replaces values of 0 with values of 2 for future division
columnList = list(toplabsDf.columns.values)
columnList.pop(columnList.index('SEQN'))

for col in columnList:
   toplabsDf[col] = toplabsDf[col].replace(0, 2, regex=True)

In [10]:
#Setting up newDem

#Renames the columns in demographicDf
demographicDf = demographicDf.rename(columns=\
                                    {
"SDDSRVYR":"Data release cycle",
"RIDSTATR":"Interview and examination status of the participant.",
"RIAGENDR":"Gender of the participant.",
"RIDAGEYR":"Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age.",
"RIDAGEMN":"Age in months of the participant at the time of screening. Reported for persons aged 24 months or younger at the time of exam (or screening if not examined).",
"RIDRETH1":"Recode of reported race and Hispanic origin information",
"RIDRETH3":"Recode of reported race and Hispanic origin information, with Non-Hispanic Asian Category",
"RIDEXMON":"Six month time period when the examination was performed - two categories: November 1 through April 30, May 1 through October 31.",
"RIDEXAGM":"Age in months of the participant at the time of examination. Reported for persons aged 19 years or younger at the time of examination.",
"DMQMILIZ":"{Have you/Has SP} ever served on active duty in the U.S. Armed Forces, military Reserves, or National Guard? (Active duty does not include training for the Reserves or National Guard, but does include activation, for service in the U.S. or in a foreign country, in support of military or humanitarian operations.)",
"DMQADFC":"Did {you/SP} ever serve in a foreign country during a time of armed conflict or on a humanitarian or peace-keeping mission? (This would include National Guard or reserve or active duty monitoring or conducting peace keeping operations in Bosnia and Kosovo, in the Sinai between Egypt and Israel, or in response to the 2004 tsunami or Haiti in 2010.)",
"DMDBORN4":"In what country {were you/was SP} born?",
"DMDCITZN":"{Are you/Is SP} a citizen of the United States? [Information about citizenship is being collected by the U.S. Public Health Service to perform health related research. Providing this information is voluntary and is collected under the authority of the Public Health Service Act. There will be no effect on pending immigration or citizenship petitions.]",
"DMDYRSUS":"Length of time the participant has been in the US.",
"DMDEDUC3":"What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?",
"DMDEDUC2":"What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?",
"DMDMARTL":"Marital status",
"RIDEXPRG":"Pregnancy status for females between 20 and 44 years of age at the time of MEC exam.",
"SIALANG":"Language of the Sample Person Interview Instrument",
"SIAPROXY":"Was a Proxy respondent used in conducting the Sample Person (SP) interview?",
"SIAINTRP":"Was an interpreter used to conduct the Sample Person (SP) interview?",
"FIALANG":"Language of the Family Interview Instrument",
"FIAPROXY":"Was a Proxy respondent used in conducting the Family Interview?",
"FIAINTRP":"Was an interpreter used to conduct the Family interview?",
"MIALANG":"Language of the MEC CAPI Interview Instrument",
"MIAPROXY":"Was a Proxy respondent used in conducting the MEC CAPI Interview?",
"MIAINTRP":"Was an interpreter used to conduct the MEC CAPI interview?",
"AIALANGA":"Language of the MEC ACASI Interview Instrument",
"DMDHHSIZ":"Total number of people in the Household",
"DMDFMSIZ":"Total number of people in the Family",
"DMDHHSZA":"Number of children aged 5 years or younger in the household",
"DMDHHSZB":"Number of children aged 6-17 years old in the household",
"DMDHHSZE":"Number of adults aged 60 years or older in the household",
"DMDHRGND":"HH reference person's gender",
"DMDHRAGE":"HH reference person's age in years",
"DMDHRBR4":"HH reference person's country of birth",
"DMDHREDU":"HH reference person's education level",
"DMDHRMAR":"HH reference person's marital status",
"DMDHSEDU":"HH reference person's spouse's education level",
"WTINT2YR":"Full sample 2 year interview weight.",
"WTMEC2YR":"Full sample 2 year MEC exam weight.",
"SDMVPSU":"Masked variance unit pseudo-PSU variable for variance estimation",
"SDMVSTRA":"Masked variance unit pseudo-stratum variable for variance estimation",
"INDHHIN2":"Total household income (reported as a range value in dollars)",
"INDFMIN2":"Total family income (reported as a range value in dollars)",
"INDFMPIR":"A ratio of family income to poverty guidelines."})

# Create a new dataframe with selected columns from the demographicDF 
newDem = demographicDf[["SEQN","Gender of the participant.", "Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age." ,"Marital status" ,"Pregnancy status for females between 20 and 44 years of age at the time of MEC exam." , "Did {you/SP} ever serve in a foreign country during a time of armed conflict or on a humanitarian or peace-keeping mission? (This would include National Guard or reserve or active duty monitoring or conducting peace keeping operations in Bosnia and Kosovo, in the Sinai between Egypt and Israel, or in response to the 2004 tsunami or Haiti in 2010.)" , "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?" , "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?"]]

#Renames the columns in newDem
newDem = newDem.rename(columns=\
                                    {"Gender of the participant.": "Gender",\
                                      "Age in years of the participant at the time of screening. Individuals 80 and over are topcoded at 80 years of age.": "Age",\
                                      "Pregnancy status for females between 20 and 44 years of age at the time of MEC exam.": "Pregnancy_Status", \
                                      "Did {you/SP} ever serve in a foreign country during a time of armed conflict or on a humanitarian or peace-keeping mission? (This would include National Guard or reserve or active duty monitoring or conducting peace keeping operations in Bosnia and Kosovo, in the Sinai between Egypt and Israel, or in response to the 2004 tsunami or Haiti in 2010.)": "Served",\
                                      "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?": "Education1",\
                                      "What is the highest grade or level of school {you have/SP has} completed or the highest degree {you have/s/he has} received?": "Education2"
                                     
                                    })

#Drops columns we don't need, such as education, military status, and pregnancy status
newDem = newDem[["SEQN", "Gender", "Age", "Marital status"]]

In [11]:
#Setting up mainDf

#Combines all three dataframes into a main dataframe
mainDf = newDem.join(newMed.set_index("SEQN"), on='SEQN', how='outer')
mainDf = mainDf.join(toplabsDf.set_index("SEQN"), on="SEQN", how="outer")

#Drops duplicate SEQN entries
mainDf = mainDf.drop_duplicates(subset='SEQN',keep='last')

#Drops any remaining rows containing null values
mainDf = mainDf.dropna()

#Replaces 2 values back to 0 values (oops!)
#columnList = list(mainDf.columns.values)

#for col in columnList:
#    mainDf[col] = mainDf[col].replace(2, 0, regex=True)

In [ ]:
#Converts mainDf into a .csv file

mainDf.to_csv("mainData.csv")